In [1]:
import re
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tikzplotlib

ImportError: cannot import name '_imaging' from 'PIL' (/usr/lib/python3/dist-packages/PIL/__init__.py)

In [ ]:
# FIXME use tv_sec

In [ ]:
enterup = "enter update"
enterdal = "enter DAL"
exitdal = "exit DAL"
exitup = "exit update"

def get_app_overheads(filename, num_warmup):
    app_send_dur = []
    app_recv_dur = []

    with open(filename) as f:
        tmp = 0
        ctr = 0
        while (line := f.readline()):
            # every four lines is a new run
            ctr += 1
            if ctr <= num_warmup:
                continue
            if (match := re.search(
                "(([0-9]+), \"([a-z]+ [a-zA-Z]+)\", Instant { tv_sec: [0-9]+, tv_nsec: ([0-9]+) })",
                line
            )): 
                run_ct = int(match.group(2))
                level = match.group(3)
                timestamp = int(match.group(4))

                if level == enterup:
                    tmp = timestamp
                elif level == enterdal:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    app_send_dur.append(dur)
                elif level == exitdal:
                    tmp = timestamp
                else:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    app_recv_dur.append(dur)
        print("ctr: ", ctr)
        print("counted: ", ctr - num_warmup)

    return app_send_dur, app_recv_dur

In [ ]:
enterpovs = "enter POVS"
enterenc = "enter ENC"
exitcore = "exit CORE"

def get_core_send_overheads(filename, num_core_send_warmup):
    pov_send_dur = []
    enc_send_dur = []

    with open(filename) as f:
        tmp = 0
        ctr = 0
        pov_run = []
        enc_run = []
        while (line := f.readline()):
            # every three lines is a new message
            ctr += 1
            if ctr <= num_core_send_warmup:
                continue
            if (match := re.search(
                "(([0-9]+), \"([a-z]+ [a-zA-Z]+)\", Instant { tv_sec: [0-9]+, tv_nsec: ([0-9]+) })",
                line
            )):
                run_ct = int(match.group(2))
                level = match.group(3)
                timestamp = int(match.group(4))

                # every 6 lines is a new run
                mod = ctr % 6
                if mod == 0:
                    # pov
                    pov_dur = 0
                    for part in pov_run:
                        pov_dur += part
                    pov_send_dur.append(pov_dur)
                    pov_run = []
                    # enc
                    enc_dur = 0
                    for part in enc_run:
                        enc_dur += part
                    enc_send_dur.append(enc_dur)
                    enc_run = []

                if level == enterpovs:
                    tmp = timestamp
                elif level == enterenc:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    pov_run.append(dur)
                    tmp = timestamp
                else:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    enc_run.append(dur)
        print("ctr: ", ctr)
        print("counted: ", ctr - num_core_send_warmup)

    return pov_send_dur, enc_send_dur

In [ ]:
enterdec = "enter DECR"
exitdec = "enter POVS"
exitcore = "exit CORE"

def get_core_recv_overheads(filename, num_core_recv_warmup):
    enc_recv_dur = []
    pov_recv_dur = []

    with open(filename) as f:
        tmp = 0
        ctr = 0
        pov_run = []
        enc_run = []
        while (line := f.readline()):
            # every three lines is a new message
            ctr += 1
            if ctr <= num_core_recv_warmup:
                continue
            if (match := re.search(
                "(([0-9]+), \"([a-z]+ [a-zA-Z]+)\", Instant { tv_sec: [0-9]+, tv_nsec: ([0-9]+) })",
                line
            )):
                run_ct = int(match.group(2))
                level = match.group(3)
                timestamp = int(match.group(4))

                # every 6 lines is a new run
                mod = ctr % 6
                if mod == 0:
                    # pov
                    pov_dur = 0
                    for part in pov_run:
                        pov_dur += part
                    pov_recv_dur.append(pov_dur)
                    pov_run = []
                    # enc
                    enc_dur = 0
                    for part in enc_run:
                        enc_dur += part
                    enc_recv_dur.append(enc_dur)
                    enc_run = []
            
                if level == enterdec:
                    tmp = timestamp
                elif level == exitdec:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    enc_run.append(dur)
                    tmp = timestamp
                else:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    pov_run.append(dur)
        print("ctr: ", ctr)
        print("counted: ", ctr - num_core_recv_warmup)

    return enc_recv_dur, pov_recv_dur

In [ ]:
enter_send_dal = "enter sendDAL"
entercore = "enter CORE"
exitcore = "exit CORE"
exit_last_core = "exit LASTCORE"
exit_send_dal = "exit sendDAL"

def get_dal_send_overheads(filename, num_dal_send_warmup):
    dal_send_dur = []
    with open(filename) as f:
        tmp = 0
        ctr = 0
        run = []
        while (line := f.readline()):
            ctr += 1
            if ctr <= num_dal_send_warmup:
                continue
            if (match := re.search(
                "(([0-9]+), \"([a-z]+ [a-zA-Z]+)\", Instant { tv_sec: [0-9]+, tv_nsec: ([0-9]+) })",
                line
            )):
                run_ct = int(match.group(2))
                level = match.group(3)
                timestamp = int(match.group(4))

                # every 6 lines is a new run
                mod = ctr % 6
                if mod == 0:
                    dur = 0
                    for part in run:
                        dur += part
                    dal_send_dur.append(dur)
                    run = []

                if level == enter_send_dal:
                    tmp = timestamp
                elif level == entercore:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    run.append(dur)
                elif level == exitcore:
                    tmp = timestamp
                elif level == exit_last_core:
                    tmp = timestamp
                else:
                    dur = timestamp - tmp
                    if dur < 0:
                        dur += 1000000000
                    run.append(dur)
        print("ctr: ", ctr)
        print("counted: ", ctr - num_dal_send_warmup)

    return dal_send_dur

In [ ]:
enter_recv_dal = "enter recvDAL"
exit_recv_dal = "exit recvDAL"

def get_dal_recv_overheads(filename_update, filename_dummy, num_dal_recv_warmup):
    dal_recv_dur_update = []
    with open(filename_update) as f:
        tmp = 0
        ctr = 0
        while (line := f.readline()):
            ctr += 1
            if ctr <= num_dal_recv_warmup:
                continue
            if (match := re.search(
                "(([0-9]+), \"([a-z]+ [a-zA-Z]+)\", Instant { tv_sec: [0-9]+, tv_nsec: ([0-9]+) })",
                line
            )):
                run_ct = int(match.group(2))
                level = match.group(3)
                timestamp = int(match.group(4))

                if level == enter_recv_dal:
                    tmp = timestamp
                else:
                    dur = timestamp - tmp;
                    if dur < 0:
                        dur += 1000000000
                    dal_recv_dur_update.append(dur)

    dal_recv_dur_dummy = []
    with open(filename_dummy) as f:
        tmp = 0
        ctr = 0
        while (line := f.readline()):
            ctr += 1
            if ctr <= num_dal_recv_warmup:
                continue
            if (match := re.search(
                "(([0-9]+), \"([a-z]+ [a-zA-Z]+)\", Instant { tv_sec: [0-9]+, tv_nsec: ([0-9]+) })",
                line
            )):
                run_ct = int(match.group(2))
                level = match.group(3)
                timestamp = int(match.group(4))
                
                if level == enter_recv_dal:
                    tmp = timestamp
                else:
                    dur = timestamp - tmp;
                    if dur < 0:
                        dur += 1000000000
                    dal_recv_dur_dummy.append(dur)

    print(len(dal_recv_dur_update))
    print(len(dal_recv_dur_dummy))

    sum_dal_recv_dur = []
    for (update_dur, dummy_dur) in zip(dal_recv_dur_update, dal_recv_dur_dummy):
        sum_dal_recv_dur.append(update_dur + dummy_dur)

    return sum_dal_recv_dur

In [ ]:
subdir = "/update_pass_output"

def app_filename(num_clients, num_runs):
    return ".{}/{}c_{}r_ts_app.txt".format(subdir, num_clients, num_runs)

def dal_send_filename(num_clients, num_runs):
    return ".{}/{}c_{}r_ts_dal_send.txt".format(subdir, num_clients, num_runs)

def dal_recv_filename_update(num_clients, num_runs):
    return ".{}/{}c_{}r_ts_dal_recv_update.txt".format(subdir, num_clients, num_runs)

def dal_recv_filename_dummy(num_clients, num_runs):
    return ".{}/{}c_{}r_ts_dal_recv_dummy.txt".format(subdir, num_clients, num_runs)

def core_send_filename(num_clients, num_runs):
    return ".{}/{}c_{}r_ts_core_send.txt".format(subdir, num_clients, num_runs)

def core_recv_filename(num_clients, num_runs):
    return ".{}/{}c_{}r_ts_core_recv.txt".format(subdir, num_clients, num_runs)


In [ ]:
warmup_count = 4
runs_count = 4
clients_count = 2

app_send_dur_0, app_recv_dur_0 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_0 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_0 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_0, enc_send_dur_0 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_0, pov_recv_dur_0 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
print(enc_send_dur_0)
print(pov_send_dur_0)
print(dal_send_dur_0)
print(app_send_dur_0)

In [ ]:
print(mean(enc_send_dur_0))
print(mean(pov_send_dur_0))
print(mean(dal_send_dur_0))
print(mean(app_send_dur_0))

In [ ]:
df = pd.DataFrame([
    ["3clients", mean(enc_send_dur_0), mean(pov_send_dur_0), mean(dal_send_dur_0), mean(app_send_dur_0)],
], columns=["update_pass", "Enc", "POV", "DAL", "App logic"])
df.plot(x='update_pass', kind='bar', stacked=True, title='Send overheads')
plt.show

In [ ]:
df = pd.DataFrame([
    ["3clients", mean(enc_recv_dur_0), mean(pov_recv_dur_0), mean(dal_recv_dur_0), mean(app_recv_dur_0)],
], columns=["update_pass", "Enc", "POV", "DAL", "App logic"])
df.plot(x='update_pass', kind='bar', stacked=True, title='Receive overheads')
plt.show

In [ ]:
warmup_count = 1000
runs_count = 10000
clients_count = 1

app_send_dur_0, app_recv_dur_0 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_0 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_0 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_0, enc_send_dur_0 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_0, pov_recv_dur_0 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
warmup_count = 1000
runs_count = 10000
clients_count = 2

app_send_dur_1, app_recv_dur_1 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_1 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_1 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_1, enc_send_dur_1 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_1, pov_recv_dur_1 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
warmup_count = 1000
runs_count = 10000
clients_count = 4

app_send_dur_2, app_recv_dur_2 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_2 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_2 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_2, enc_send_dur_2 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_2, pov_recv_dur_2 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
warmup_count = 1000
runs_count = 10000
clients_count = 8

app_send_dur_4, app_recv_dur_4 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_4 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_4 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_4, enc_send_dur_4 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_4, pov_recv_dur_4 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
warmup_count = 1000
runs_count = 10000
clients_count = 16

app_send_dur_5, app_recv_dur_5 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_5 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_5 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_5, enc_send_dur_5 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_5, pov_recv_dur_5 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
warmup_count = 1000
runs_count = 10000
clients_count = 32

app_send_dur_6, app_recv_dur_6 = get_app_overheads(app_filename(clients_count, runs_count), 4 * warmup_count)
dal_send_dur_6 = get_dal_send_overheads(dal_send_filename(clients_count, runs_count), 6 * warmup_count)
dal_recv_dur_6 = get_dal_recv_overheads(dal_recv_filename_update(clients_count, runs_count), dal_recv_filename_dummy(clients_count, runs_count), 2 * warmup_count)
pov_send_dur_6, enc_send_dur_6 = get_core_send_overheads(core_send_filename(clients_count, runs_count), 3 * 2 * warmup_count)
enc_recv_dur_6, pov_recv_dur_6 = get_core_recv_overheads(core_recv_filename(clients_count, runs_count), 3 * 2 * warmup_count)

In [ ]:
print(mean(app_send_dur_1))
print(mean(app_send_dur_2))
print(mean(pov_send_dur_1))
print(mean(pov_send_dur_2))
print(mean(enc_send_dur_1))
print(mean(enc_send_dur_2))

In [ ]:
print(mean(enc_recv_dur_1))
print(mean(enc_recv_dur_2))
print(mean(pov_recv_dur_1))
print(mean(pov_recv_dur_2))
print(mean(app_recv_dur_1))
print(mean(app_recv_dur_2))

In [ ]:
df = pd.DataFrame([
    ["1clients", mean(enc_send_dur_0), mean(pov_send_dur_0), mean(dal_send_dur_0), mean(app_send_dur_0)],
    ["2clients", mean(enc_send_dur_1), mean(pov_send_dur_1), mean(dal_send_dur_1), mean(app_send_dur_1)],
    ["4clients", mean(enc_send_dur_2), mean(pov_send_dur_2), mean(dal_send_dur_2), mean(app_send_dur_2)],
    ["8clients", mean(enc_send_dur_4), mean(pov_send_dur_4), mean(dal_send_dur_4), mean(app_send_dur_4)],
    ["16clients", mean(enc_send_dur_5), mean(pov_send_dur_5), mean(dal_send_dur_5), mean(app_send_dur_5)],
    ["32clients", mean(enc_send_dur_6), mean(pov_send_dur_6), mean(dal_send_dur_6), mean(app_send_dur_6)],
], columns=["update_pass", "Enc", "POV", "DAL", "App logic"])
df.plot(x='update_pass', kind='bar', stacked=True, title='Send overheads')
plt.show
tikzplotlib.save("test.tex")

In [ ]:
df = pd.DataFrame([
    ["1clients", mean(enc_recv_dur_0), mean(pov_recv_dur_0), mean(dal_recv_dur_0), mean(app_recv_dur_0)],
    ["2clients", mean(enc_recv_dur_1), mean(pov_recv_dur_1), mean(dal_recv_dur_1), mean(app_recv_dur_1)],
    ["4clients", mean(enc_recv_dur_2), mean(pov_recv_dur_2), mean(dal_recv_dur_2), mean(app_recv_dur_2)],
    ["8clients", mean(enc_recv_dur_4), mean(pov_recv_dur_4), mean(dal_recv_dur_4), mean(app_recv_dur_4)],
    ["16clients", mean(enc_recv_dur_5), mean(pov_recv_dur_5), mean(dal_recv_dur_5), mean(app_recv_dur_5)],
    ["32clients", mean(enc_recv_dur_6), mean(pov_recv_dur_6), mean(dal_recv_dur_6), mean(app_recv_dur_6)],
], columns=["update_pass", "Enc", "POV", "DAL", "App logic"])
df.plot(x='update_pass', kind='bar', stacked=True, title='Receive overheads')
plt.show